# 🏥 Complete Medical Intelligence System with Vector Search

This notebook implements a comprehensive medical analysis system using BigFrames and Google Cloud technologies. The system is designed to provide intelligent patient analysis, homeopathic medicine recognition, and advanced AI-powered medical insights.

## 🎯 **System Overview**

Our medical intelligence system combines cutting-edge AI technologies to create a powerful tool for:
- **Semantic Patient Search**: Find similar patients using vector embeddings
- **Medical Prescription Analysis**: Decode homeopathic medicine abbreviations
- **Intelligent Recommendations**: AI-powered health insights and suggestions

## 🔧 **Technologies Used**

| Technology | Purpose | Capability |
|------------|---------|------------|
| `bigframes.ml.llm.TextEmbeddingGenerator` | Generate semantic embeddings | Vector representations of patient data |
| `bigframes.bigquery.vector_search` | Semantic similarity search | Find similar patients by medical conditions |
| `bigframes.ml.llm.GeminiTextGenerator` | AI-powered analysis | Generate medical insights and recommendations |


## 📚 **Notebook Structure**

This notebook is organized into **3 comprehensive modules**, each building upon the previous one:

### **Core BigFrames Implementation**
**Foundation Layer** - Essential vector search and AI capabilities
- Environment setup and data processing
- Vector embeddings generation and storage
- Basic semantic search functionality
- AI analysis and forecasting setup


## 📦 Step 1: Install Required Packages


In [ ]:
# Install required packages
!pip install bigframes pandas google-cloud-bigquery

# If running in Colab, authenticate
try:
    from google.colab import auth
    auth.authenticate_user()
    print("✅ Colab authentication completed")
except ImportError:
    print("✅ Running outside Colab - using local credentials")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.1 MB/s eta 0:00:00
✅ Colab authentication completed


## ⚙️ Step 2: Configuration and Setup


In [ ]:
import os
import pandas as pd
import bigframes.pandas as bpd
import bigframes
import bigframes.bigquery as bbq
from google.cloud import bigquery
from bigframes.ml.llm import TextEmbeddingGenerator, GeminiTextGenerator
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration - UPDATE YOUR PROJECT ID HERE
PROJECT_ID = "thinking-bonbon-471314-i4"  # 🔄 CHANGE THIS TO YOUR PROJECT ID
LOCATION = "US"
DATASET_NAME = "patients_vector_search_demo"
TABLE_NAME = "patients_with_embeddings"
FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}"
EMBEDDING_TABLE_ID = f"{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}_embeddings"

# Configure BigFrames
bigframes.options.bigquery.project = PROJECT_ID
bigframes.options.bigquery.location = LOCATION

print(f"🎯 Project: {PROJECT_ID}")
print(f"📊 Dataset: {DATASET_NAME}")
print(f"🗃️ Table: {TABLE_NAME}")
print(f"🔍 Embedding Table: {EMBEDDING_TABLE_ID}")
print(f"✅ BigFrames configured successfully!")


🎯 Project: thinking-bonbon-471314-i4
📊 Dataset: patients_vector_search_demo
🗃️ Table: patients_with_embeddings
🔍 Embedding Table: thinking-bonbon-471314-i4.patients_vector_search_demo.patients_with_embeddings_embeddings
✅ BigFrames configured successfully!


## 📁 Step 3: Load Your Patient Data

Upload your `synthetic_500_patients.csv` file when prompted below.


In [ ]:
! pip install kagglehub[pandas-datasets]

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# 🗂️ Path to the CSV file inside the dataset
file_path = "synthetic_500_patients.csv"

# 📥 Load the latest version of the dataset as a pandas DataFrame
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "yashwanthkrishna78/patient-file-database",
    file_path,
)

# ✅ Show confirmation and summary
print("✅ Successfully loaded from Kaggle Hub\n")
print(f"📊 Dataset Shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}\n")

print("🔍 Data Types:")
print(df.dtypes, "\n")


print("📊 Descriptive stats (numeric):")
print(df.describe(include=['number']).T, "\n")

print("📊 Descriptive stats (text):")
print(df.describe(include=['object']).T, "\n")

print("📝 First 5 rows:")
print(df.head())


Using Colab cache for faster access to the 'patient-file-database' dataset.
✅ Successfully loaded from Kaggle Hub

📊 Dataset Shape: (500, 8)
📋 Columns: ['PID', 'FirstName', 'LastName', 'Age', 'Address', 'FirstVisit', 'Gender', 'Prescriptions']

🔍 Data Types:
PID               int64
FirstName        object
LastName         object
Age               int64
Address          object
FirstVisit       object
Gender           object
Prescriptions    object
dtype: object 

📊 Descriptive stats (numeric):
     count     mean         std  min     25%    50%     75%    max
PID  500.0  250.500  144.481833  1.0  125.75  250.5  375.25  500.0
Age  500.0   47.088   21.252378  1.0   30.00   49.0   66.00   80.0 

📊 Descriptive stats (text):
              count unique                                                top  \
FirstName       500     50                                            Deepika   
LastName        500     40                                                Rao   
Address         500    500  

## 🧠 Step 4: Prepare Data and Generate Embeddings


In [ ]:
# Create rich patient descriptions for better embeddings
def create_patient_description(row):
    return (f"Patient ID: {row['PID']} - Patient {row['FirstName']} {row['LastName']} is a {row['Age']} year old "
            f"{row['Gender'].lower()} patient living at {row['Address']}. "
            f"Medical prescriptions: {row['Prescriptions']}. "
            f"First visit: {row['FirstVisit']}")

df['patient_description'] = df.apply(create_patient_description, axis=1)

print(f"✅ Created patient descriptions")
print(f"\n📄 Sample description:")
print(df['patient_description'].iloc[0][:200] + "...")

# Create BigQuery dataset and upload data
client = bigquery.Client(project=PROJECT_ID)
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_NAME}")

try:
    client.get_dataset(dataset_ref)
    print(f"✅ Dataset {DATASET_NAME} exists")
except Exception:
    dataset_ref.location = LOCATION
    client.create_dataset(dataset_ref, exists_ok=True)
    print(f"✅ Created dataset {DATASET_NAME}")

# Upload data to BigQuery
bdf = bpd.DataFrame(df)
bdf.to_gbq(FULL_TABLE_ID, if_exists="replace")
print(f"✅ Uploaded data to {FULL_TABLE_ID}")


✅ Created patient descriptions

📄 Sample description:
Patient ID: 1 - Patient Shanti Mehta is a 55 year old m patient living at 388, Temple Road, Kalyan. Medical prescriptions: 11/14/16 00:00:00 - thy 30  lssl cp 6x 44     100 |--| 12/16/15 00:00:00 - pe...
✅ Dataset patients_vector_search_demo exists


✅ Uploaded data to thinking-bonbon-471314-i4.patients_vector_search_demo.patients_with_embeddings


In [ ]:
# Generate embeddings using TextEmbeddingGenerator with text-multilingual-embedding-002
print("🧠 Generating embeddings for all patients...")
print("⏳ This may take a few minutes...")

start_time = time.time()

# Initialize TextEmbeddingGenerator with the specific model
text_model = TextEmbeddingGenerator()

# Read data from BigQuery
bdf_remote = bpd.read_gbq(FULL_TABLE_ID)

# Prepare data for indexing - create content column from patient_description
df_to_index = bdf_remote.assign(content=bdf_remote["patient_description"])

# Filter out empty content
df_to_index = df_to_index[df_to_index["content"] != ""]

# Generate embeddings
embedding = text_model.predict(df_to_index)

# Check the status column to look for errors
successful_rows = (embedding['ml_generate_embedding_status'] == '').sum()
failed_rows = (embedding['ml_generate_embedding_status'] != '').sum()
print(f"✅ Successful rows: {successful_rows}")
print(f"❌ Failed rows: {failed_rows}")
print(f"📊 Embedding shape: {embedding.shape}")

# Save embeddings to table
embedding_table_id = f"{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}_embeddings"
embedding.to_gbq(embedding_table_id, if_exists="replace")

end_time = time.time()
print(f"✅ Generated embeddings in {end_time - start_time:.1f} seconds")
print(f"📊 Embedding columns: {embedding.columns.tolist()}")
print(f"💾 Saved embeddings to {embedding_table_id}")


🧠 Generating embeddings for all patients...
⏳ This may take a few minutes...


Requested cancellation for Query job ed192861-3978-4720-967a-3c7915c454e0 in location US...


KeyboardInterrupt: 

## 🔍 Step 5: Vector Search Functions


In [ ]:
def ask_question(question, top_k=5):
    """
    Ask a question and get similar patients using bigframes vector search
    Also handles direct PID queries like 'PID 321' or 'tell me about patient 321'
    """
    print(f"\n🔍 Searching for: '{question}'")

    # Check if this is a PID query
    import re
    pid_match = re.search(r'(?:pid|patient)\s*(\d+)', question.lower())

    if pid_match:
        # Direct PID lookup from the embeddings table
        pid_number = pid_match.group(1)
        print(f"🎯 Direct PID lookup for Patient ID: {pid_number}")

        try:
            # Read from embeddings table and filter by PID
            df_written = bpd.read_gbq(EMBEDDING_TABLE_ID)
            results = df_written[df_written["PID"] == int(pid_number)]

            if len(results) == 0:
                print(f"❌ No patient found with PID {pid_number}")
                return None

            print(f"✅ Found patient with PID {pid_number}:")

            # Convert to pandas for display
            results_pd = results.to_pandas()

            for i, row in results_pd.iterrows():
                print(f"\n--- Patient Details ---")
                print(f"🆔 PID: {row['PID']}")
                print(f"👤 {row['FirstName']} {row['LastName']}, Age {row['Age']}, {row['Gender']}")
                print(f"🏠 {row['Address']}")
                print(f"📅 First Visit: {row['FirstVisit']}")
                print(f"💊 Full Prescriptions: {row['Prescriptions']}")

            return results_pd

        except Exception as e:
            print(f"❌ PID search error: {e}")
            return None

    else:
        # Semantic vector search using bigframes.bigquery.vector_search
        try:
            # Generate embedding for the search string
            text_model = TextEmbeddingGenerator(model_name="text-multilingual-embedding-002")
            search_df = bpd.DataFrame([question], columns=['search_string'])
            search_embedding = text_model.predict(search_df)

            # Perform vector search using bigframes
            vector_search_results = bbq.vector_search(
                base_table=EMBEDDING_TABLE_ID,
                column_to_search="ml_generate_embedding_result",
                query=search_embedding,
                distance_type="COSINE",
                query_column_to_search="ml_generate_embedding_result",
                top_k=top_k,
            )

            # Get results and convert to pandas for display
            results = vector_search_results[["PID", "FirstName", "LastName", "Age", "Gender", "Address", "Prescriptions", "distance"]].sort_values("distance")
            results_pd = results.to_pandas()

            print(f"✅ Found {len(results_pd)} similar patients:")

            for i, row in results_pd.iterrows():
                similarity_percent = round((1 - row['distance']) * 100, 1)
                prescriptions_preview = str(row['Prescriptions'])[:100] if row['Prescriptions'] else "No prescriptions"

                print(f"\n--- Match {i+1} ({similarity_percent}% similar) ---")
                print(f"🆔 PID: {row['PID']}")
                print(f"👤 {row['FirstName']} {row['LastName']}, Age {row['Age']}, {row['Gender']}")
                print(f"🏠 {row['Address']}")
                print(f"💊 {prescriptions_preview}...")

            return results_pd

        except Exception as e:
            print(f"❌ Search error: {e}")
            return None

print("✅ Vector search function ready with bigframes vector_search and PID support!")


✅ Vector search function ready with bigframes vector_search and PID support!


## 🎯 Step 6: Test Vector Search with Medical Queries


In [ ]:
# Test vector search with medical queries
test_queries = [
    "Get the medical pricption of Shobha,Agarwal",
]

print("🎯 TESTING VECTOR SEARCH WITH MEDICAL QUERIES")
print("=" * 60)

search_results = []
for query in test_queries:
    results = ask_question(query, top_k=3)

    search_results.append(results)
    print("\n" + "-" * 40)

print("\n✅ Vector search testing completed!")


🎯 TESTING VECTOR SEARCH WITH MEDICAL QUERIES

🔍 Searching for: 'Get the medical pricption of Shobha,Agarwal'


✅ Found 3 similar patients:

--- Match 1 (78.3% similar) ---
🆔 PID: 431
👤 Shobha Agarwal, Age 41, M
🏠 807, Hospital Road, Jaipur
💊 07/04/22 00:00:00 - 1,2 / 40 size sl sl cp+sil 6x 2 2 // kalibrome 200 hd // 3,4 // 40 size sl sl cp...

--- Match 1 (72.1% similar) ---
🆔 PID: 443
👤 Shobha Jindal, Age 67, M
🏠 451, Station Road, Coimbatore
💊 02/28/22 00:00:00 - 40 size sl sl echina q 15 15 // sl hd // 15 days 270 extra ;- wheezal soaps  |--...

--- Match 1 (71.8% similar) ---
🆔 PID: 447
👤 Shobha Shah, Age 77, F
🏠 371, Market Street, Faridabad
💊 12/17/24 00:00:00 - migraine   month pain...

----------------------------------------

✅ Vector search testing completed!


## 🤖 Step 7: AI Analysis with GeminiTextGenerator


In [ ]:
def get_ai_analysis(search_results, question):
    """
    Get AI analysis of search results using GeminiTextGenerator with enhanced medical prompt
    """
    if search_results is None or len(search_results) == 0:
        return None

    print(f"\n🤖 Getting AI analysis for: '{question}'")

    try:
        # Prepare patient summaries for AI analysis
        patient_summaries = []
        for _, row in search_results.head(3).iterrows():
            summary = (f"Patient ID: {row.get('PID', 'N/A')} - {row.get('FirstName', 'Unknown')} {row.get('LastName', 'Unknown')}, "
                      f"Age {row.get('Age', 'N/A')}, Gender {row.get('Gender', 'N/A')}, "
                      f"Address: {row.get('Address', 'N/A')}, "
                      f"Prescriptions: {row.get('Prescriptions', 'N/A')}")
            patient_summaries.append(summary)

        # Enhanced analysis prompt with homeopathic medicine knowledge
        analysis_prompt = f"""
        🏥 HOMEOPATHIC MEDICAL ANALYSIS

        Medical Query: "{question}"

        📋 PRESCRIPTION UNDERSTANDING:
        - "|--|" separates different prescription visits/dates
        - Common homeopathic abbreviations: thy=Thyroidinum, arn=Arnica, bry=Bryonia, lssl=Lycopodium, cp=Carcinosin, mp=Magnesia Phosphorica, np=Natrum Phosphoricum, cf=Calcarea Fluorica, sl=Sac Lac
        - Potencies: 30, 200c, 6x indicate medicine strength/dilution levels
        - bid=twice daily, tid=three times daily

        🔍 MEDICINE-CONDITION MAPPING:
        - thy (Thyroidinum) → thyroid disorders, metabolism issues
        - arn (Arnica) → trauma, bruises, muscle soreness
        - bry (Bryonia) → dry cough, headaches, joint pain
        - lssl (Lycopodium) → digestive issues, liver problems
        - cp (Carcinosin) → constitutional remedy for chronic conditions
        - mp (Magnesia Phos) → muscle cramps, neuralgic pain
        arn: Arnica Montana
        bry: Bryonia Alba
        aco: Aconitum Napellus
        ruta: Ruta Graveolens
        phyto: Phytolacca Decandra
        sulfo: Sulphur
        sul: Sulphur
        fp: Ferrum Phosphoricum
        nm: Natrum Muriaticum
        chame: Chamomilla
        sl: Sac Lac
        thy: Thyroidinum
        lssl: Lycopodium Clavatum
        cp: Carcinosin
        mp: Magnesia Phosphorica
        np: Natrum Phosphoricum
        kp: Kali Phosphoricum
        kpcf: Kali Phosphoricum
        tmv: Tuberculinum
        nux: Nux Vomica
        apis: Apis Mellifica
        ylox: Yohimbinum
        drom: Drosera Rotundifolia
        bid: Twice daily
        cf: Calcarea Fluorica
        lba: Lachesis
        rt: Right side
        cbp: Chronic Blood Pressure
        rbs: Random Blood Sugar
        cc: Carbo Vegetabilis
        apd: Argentum Phosphoricum

        👥 TOP MATCHING PATIENTS:
        {chr(10).join(patient_summaries)}

        📊 PROVIDE ANALYSIS:
        1. What homeopathic medicines appeared?
        2. What medical conditions do these medicines suggest?
        3. What insights can you provide about treatment approaches?
        4. Any recommendations for similar cases?

        Keep the analysis practical and focused on medical insights that would help a homeopathic practitioner.
        """

        # Use GeminiTextGenerator
        gemini = GeminiTextGenerator()
        prompt_df = bpd.DataFrame({"prompt": [analysis_prompt]})
        response = gemini.predict(prompt_df)
        analysis = response.to_pandas().iloc[0, 0]

        print(f"✅ AI Analysis:")
        print(f"{analysis}")
        return analysis

    except Exception as e:
        print(f"⚠️ AI analysis error: {e}")
        print("Note: GeminiTextGenerator may require additional setup or permissions")
        return None

# Test AI analysis on the first search result
if search_results and search_results[0] is not None:
    ai_analysis = get_ai_analysis(search_results[0], test_queries[0])
else:
    print("⚠️ No search results available for AI analysis")


NameError: name 'search_results' is not defined

## 🎯 Step 8: Interactive Query Interface


🎯 CUSTOM QUERY EXAMPLES




## 🚀 Step 9: Direct Query Without Reprocessing


In [ ]:
# Verify embeddings are stored and ready for direct queries
def check_embeddings_status():
    """
    Check if embeddings are ready for direct queries
    """
    client = bigquery.Client(project=PROJECT_ID)
    query = f"""
    SELECT
        COUNT(*) as total_patients,
        COUNT(ml_generate_embedding_result) as patients_with_embeddings,
        COUNT(CASE WHEN ml_generate_embedding_status = '' THEN 1 END) as successful_embeddings
    FROM `{EMBEDDING_TABLE_ID}`
    """

    try:
        result = client.query(query).to_dataframe()
        total = result['total_patients'].iloc[0]
        with_embeddings = result['patients_with_embeddings'].iloc[0]
        successful = result['successful_embeddings'].iloc[0]

        print(f"📊 Dataset Status:")
        print(f"   Total patients: {total}")
        print(f"   Patients with embeddings: {with_embeddings}")
        print(f"   Successful embeddings: {successful}")
        print(f"   Ready for direct queries: {'✅ YES' if successful > 0 else '❌ NO'}")

        return successful > 0
    except Exception as e:
        print(f"❌ Error checking embeddings status: {e}")
        print("   Embeddings table may not exist yet. Run the embedding generation step first.")
        return False

# Check status
embeddings_ready = check_embeddings_status()

if embeddings_ready:
    print(f"\n🎉 SUCCESS! Your vector search system is fully operational!")
    print(f"\n📝 You can now:")
    print(f"   • Ask medical questions using query_patients()")
    print(f"   • Get AI analysis of search results")
    print(f"   • Perform time series forecasting")
    print(f"   • All without reprocessing embeddings!")
else:
    print(f"\n⚠️ Embeddings not ready. Please run the embedding generation steps.")


📊 Dataset Status:
   Total patients: 500
   Patients with embeddings: 500
   Successful embeddings: 500
   Ready for direct queries: ✅ YES

🎉 SUCCESS! Your vector search system is fully operational!

📝 You can now:
   • Ask medical questions using query_patients()
   • Get AI analysis of search results
   • Perform time series forecasting
   • All without reprocessing embeddings!


## 🎯 Step 10: Try Your Own Queries!


In [ ]:
# Interactive query function for custom questions
def query_patients(question, top_k=5, include_ai_analysis=True):
    """
    Complete query function with vector search and optional AI analysis
    """
    print(f"\n{'='*60}")
    print(f"🔍 PATIENT SEARCH QUERY")
    print(f"{'='*60}")

    # Perform vector search
    results = ask_question(question, top_k=top_k)

    # Get AI analysis if requested and results exist
    if include_ai_analysis and results is not None:
        ai_analysis = get_ai_analysis(results, question)

    return results


In [ ]:
# Try your own custom query here!
# Modify the question below and run this cell

YOUR_QUESTION = "get the medication details of pid 332"  # 🔄 CHANGE THIS TO YOUR QUESTION

print(f"🔍 YOUR CUSTOM QUERY")
print(f"Question: {YOUR_QUESTION}")
print(f"=" * 50)

# Run your query
your_results = query_patients(YOUR_QUESTION, top_k=5, include_ai_analysis=True)

print(f"\n✅ Query completed! Found {len(your_results) if your_results is not None else 0} results.")


🔍 YOUR CUSTOM QUERY
Question: get the medication details of pid 332

🔍 PATIENT SEARCH QUERY

🔍 Searching for: 'get the medication details of pid 332'
🎯 Direct PID lookup for Patient ID: 332
✅ Found patient with PID 332:

--- Patient Details ---
🆔 PID: 332
👤 Lata Jain, Age 68, F
🏠 74, Bus Stand, Bangalore
📅 First Visit: 05/31/23 08:15:11
💊 Full Prescriptions: 03/18/14 00:00:00 - pain bellow feet arn 30a nd acofp 30  ry 30 wt65kg  |--| 10/12/19 00:00:00 - c/o constipation , jp ++ // 40 size nux 30 sl sul 30 tid / 1 drtom y lox 1/2 // 15 days 250 |--| 11/01/19 00:00:00 - bry 30 gells 30 / mp 6x 4  4// sang 200 hd // 280 |--| 06/21/21 00:00:00 - 40 size sl sl km6x 2 2 // cp 30 hd wekly // one month 430 extra ;- shanti alfa syp |--| 04/29/25 00:00:00 - skin rash |--| 02/26/18 00:00:00 - middile stodal 3030  |--| 05/11/23 00:00:00 - skin rash good / wrist pain / elbow pain  |--| 10/03/23 00:00:00 - 40 size miristica 30 / ks+cs6x afternoon 4 / pyonia 30 tid // sarsa q 15 15 // ( 10 grams ) gu

✅ AI Analysis:
Okay, here's an analysis of Lata Jain's prescriptions, focusing on medical insights for a homeopathic practitioner:

**Analysis of Lata Jain's Homeopathic Treatment (Patient ID: 332)**

**1. Homeopathic Medicines Prescribed:**

*   **03/18/14:** Arnica Montana (arn 30a), Aconitum Napellus/Ferrum Phosphoricum (acofp 30), Bryonia (ry 30)
*   **10/12/19:** Nux Vomica (nux 30), Sulphur (sul 30), Drosera Rotundifolia (drom), Yohimbinum (ylox), Sac Lac (sl)
*   **11/01/19:** Bryonia (bry 30), Gelsemium Sempervirens (gells 30), Magnesia Phosphorica (mp 6x), Sanguinaria Canadensis (sang 200 hd)
*   **06/21/21:** Kali Muriaticum (km6x), Carcinosin (cp 30 hd), Sac Lac (sl), Alfa Syp (likely alfalfa syrup - not a homeopathic medicine, but a tonic)
*   **10/03/23:** Miristica Sebifera (miristica 30), Ks+Cs6x (likely a combination remedy, needs clarification of components), Paeonia Officinalis (pyonia 30), Sarsaparilla (sarsa q), Gunpowder (likely a low potency preparation, origin ne

---

 ## **Core BigFrames Implementation**

---


#### 🏗️ **Infrastructure Setup**
-  **BigFrames Environment**: Fully configured for medical data processing
-  **BigQuery Integration**: Cloud data warehouse ready for large-scale operations
-  **Authentication**: Secure access to Google Cloud services

#### 📊 **Data Processing**
-  **Patient Data Loading**: 500+ patient records with rich descriptions
-  **Vector Embeddings**: Semantic representations generated and stored
-  **Data Quality**: Clean, structured patient information ready for analysis

#### 🔍 **Search Capabilities**
-  **Vector Search**: Semantic similarity matching for patient queries
-  **AI Analysis**: GeminiTextGenerator integration for intelligent insights
-  **Direct Queries**: No need to reprocess data for future searches
